# Load Packages


In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns
from sklearn import metrics
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical, normalize
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix  
from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())
tf.config.list_physical_devices('GPU') 
print(tf.test.is_built_with_cuda())
with tf.compat.v1.Session() as sess:
  devices = sess.list_devices()
devices

True


[_DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 268435456, -1950203687394830274)]

# Import the dataset

In [ ]:
df = pd.read_csv ("glass.csv")
df2 = df.set_index('Type')
df2.head()
print (df2.head(10))

# Checking Data Types

In [ ]:
print (df.dtypes)

# checking rows for duplicate data

In [ ]:
duplicate_rows_df = df[df.duplicated()]
print("number of duplicated rows: ", duplicate_rows_df.shape)

# count the number of null values in each column

In [ ]:
print(df2.isnull().sum())

# print summary statictics

In [ ]:
print(df2.describe())

# Create a pairplot of the data

In [ ]:
p=sns.pairplot(df, hue='Type')

# Extract Labels to a different data frame.

In [ ]:
x = df.drop('Type', axis = 1)
y = df['Type']

# Plot the training set

In [ ]:
glass= y.unique()
values = y.value_counts()

plt.bar(glass, values)
plt.title("Glass Types")
plt.xlabel("Glass Types")
plt.ylabel("Frequency")
plt.show()

# explore data values

In [ ]:
x.describe()

# convert data to use with keras for normalisation

In [ ]:
x = df.values
x = normalize(x)
print (x[0])

# convert label 

In [ ]:
y = to_categorical(y)
y.shape

# Split the data for training

In [ ]:
x, x_val, y, y_val = train_test_split(x, y, test_size=0.2)
x_val, x_test, y_val, y_test = train_test_split(x_val, y_val, test_size=0.5)

# visualise the data in a heatmap

In [ ]:
corr =df2.corr()

fig, ax =plt.subplots(figsize=(10,8))
#Generate Heat Map, allow annotations and place floats in map
sns.heatmap(corr, cmap='coolwarm', annot=True, fmt=".2f")
#Apply xticks
plt.xticks(range(len(corr.columns)), corr.columns);
#Apply yticks
plt.yticks(range(len(corr.columns)), corr.columns)
#show plot
plt.show()

# Create the model

In [ ]:
model = tf.keras.models.Sequential([

    tf.keras.layers.Dense(256, input_shape=(10,), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.3),
    
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.3),
    
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),
            
    tf.keras.layers.Dense(8, activation='softmax')
])


model.compile(loss='categorical_crossentropy',
              optimizer=Adam(0.0001),
              metrics=['acc'])

model
    

# view the model

In [ ]:
model.summary()

# train the model

In [ ]:


history = model.fit(x, y,
                    epochs=100000,
                    validation_data=(x_val, y_val),
                    verbose=2,
                   )



# Evaluate the model

# Display Model Accuracy

In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# Display Model Accuracy

In [ ]:


print ("Accuracy:", history.history['acc'][-1])
print ("Validation Accuracy:", history.history['val_acc'][-1])


# Display model loss

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# Evaluate on the test data

In [ ]:
model.evaluate(x_test, y_test)

# Display the best model accuracy

In [ ]:
best_model_accuracy = history.history['acc'][np.argmin(history.history['loss'])]
print ("Best Model Accuracy:", best_model_accuracy)